# Trabajo Práctico 2: Entrenamiento y evaluación de modelos #

## 1. Métrica ##

Las métricas que vamos a utilizar son **F-SCORE** y además **Re-Call**.

### Por qué F-SCORE? ###
La misma consiste en combinar precission y recall. Los valores estan cerca de 1, cuando precissión y recall son buenos, en cambio, F-SCORE comienza a caer si uno de los dos cae.

### Por qué Re-Call? ###
Re-Call nos permite medir que no se nos escapen interesados a las tarjetas de crédito, y en caso que NO esté interesado, el rechazo de la tarjeta, no afecta al negocio. En cambio si se escapa un caso de un cliente con interés, la empresa perdería la posibilidad de generar ingresos.

## Librerías ##


In [1]:
%matplotlib inline
import warnings
import numpy as np
import pandas as pd 
import matplotlib
import keras
import h5py
import PIL
import seaborn as sns
import sklearn
import pytz
import plotly.graph_objects as go

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

# One Hot Encoder
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from sklearn.pipeline import Pipeline

pd.set_option('display.float_format',lambda x:'%.0f'% x) #Sacar notación científica en pandas
pd.options.display.max_columns = 0

plt.rcParams.update({
    "font.family": ["serif"],
    "font.sans-serif": ["Roboto"],
    "font.size": 9,
    "axes.labelsize": 11,
    "axes.titlesize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    'figure.figsize': (15.0, 4.0),
    'axes.grid': False,
    'axes.spines.left': True,
    'axes.spines.right': True,
    'axes.spines.top': True,
    'axes.spines.bottom': True,
})

np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

2022-04-20 18:44:19.715306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 18:44:19.715338: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Train ##


In [2]:
data_TC = pd.read_csv('train.csv')

BETTER_COLUMN_NAMES = {
    'ID': 'id',
    'Gender': 'sexo',
    'Age': 'edad',
    'Region_Code': 'codigo_region',
    'Occupation': 'ocupacion',
    'Channel_Code': 'codigo_canal',
    'Vintage': 'antiguedad',
    'Credit_Product': 'tiene_producto_credito_activo',
    'Avg_Account_Balance': 'saldo_promedio_cuenta',
    'Is_Active': 'es_activo',
    'Is_Lead': 'esta_interesado',
}
data_TC.rename(columns=BETTER_COLUMN_NAMES, inplace=True)

data_TC.set_index('id', inplace=True)


## Test ##


In [3]:
data_Test = pd.read_csv('test.csv')

BETTER_COLUMN_NAMES2 = {
    'ID': 'id',
    'Gender': 'sexo',
    'Age': 'edad',
    'Region_Code': 'codigo_region',
    'Occupation': 'ocupacion',
    'Channel_Code': 'codigo_canal',
    'Vintage': 'antiguedad',
    'Credit_Product': 'tiene_producto_credito_activo',
    'Avg_Account_Balance': 'saldo_promedio_cuenta',
    'Is_Active': 'es_activo',
    'Is_Lead': 'esta_interesado',
}
data_Test.rename(columns=BETTER_COLUMN_NAMES2, inplace=True)

data_Test.set_index('id', inplace=True)

In [4]:
data_Test.sample(5)

,sexo,edad,codigo_region,ocupacion,codigo_canal,antiguedad,tiene_producto_credito_activo,saldo_promedio_cuenta,es_activo
id,,,,,,,,,
UTYPZZD6,Male,45,RG282,Self_Employed,X2,21,No,537933,No
XU2QU7EA,Male,56,RG280,Self_Employed,X3,104,Yes,468255,Yes
VGR2IXGZ,Female,52,RG254,Self_Employed,X3,69,No,1843109,No
9VXUBQBR,Female,50,RG283,Self_Employed,X3,117,Yes,2417106,No
KYIOVKCD,Male,54,RG268,Self_Employed,X3,57,Yes,1653744,No


## 2. Técnica feature engineering ##

- **Redondeo de números(saldo promedio de cuenta)** Mucha información númerica puede causar sobreentrenamiento o complicarnos para encontrar límites. {VER/CONTINUAR}